# Imports

In [2]:
import $ivy.`com.github.julien-truffaut::monocle-core:1.5.0`
import $ivy.`com.github.julien-truffaut::monocle-law:1.5.0`
import $ivy.`com.github.julien-truffaut::monocle-macro:1.5.0`
import $ivy.`org.scalaz::scalaz-core:7.2.25`

import $ivy.$                                               

import $ivy.$                                              

import $ivy.$                                                

import $ivy.$                               

# Optics: a hands-on introduction in Scala
![Unicorn glasses](unicorn_glasses.jpg "Unicorn glasses")
![Bestmile logo](bm_logo.png "Bestmile logo")


*Optics*: purely functional abstractions to manipulate immutable objects

## Here's a dummy &#128118; domain model: 

In [24]:
case class Street(number: Int, name: String); 
case class Address(city: String, street: Street); 
case class Company(name: String, address: Address); 
case class Employee(name: String, company: Company, salary: Int)

defined class Street
defined class Address
defined class Company
defined class Employee

Case classes in Scala are used to define immutable and algebraic data types. They have built-in support for:
- pattern matching
- comparison by structure
- `copy` method which helps for immutable updates

Let's define an employee &#128105;:

In [25]:
val employee = Employee("Monica", Company("Bestmile", Address("Lausanne", Street(58, "rhodanie"))), 100)

employee: Employee = Employee(Monica,Company(Bestmile,Address(Lausanne,Street(58,rhodanie))),100)

Now let's say we want to upper case the street name. Here's how we can do it using `copy`:

In [5]:
employee.copy(
  company = employee.company.copy(
    address = employee.company.address.copy(
      street = employee.company.address.street.copy(
        name = employee.company.address.street.name.capitalize // luckily capitalize exists
      )
    )
  )
)

res4: Employee = Employee(Monica,Company(Bestmile,Address(Lausanne,Street(58,Rhodanie))),100)

Pretty convoluted &#128580;. That's when the `Lens` optic becomes useful, as we'll see next.

## `Lens`: magnifying glass  &#128269; for immutable data structures

`Lens` is like a zoom for data structures. Let's import the `monocle` library version of `Lens` first (we'll look at `scalaz` later):

In [6]:
import monocle.Lens
import monocle.macros.GenLens

import monocle.Lens

import monocle.macros.GenLens

Let's define a `lens` on the `company` field of `Employee`:

In [7]:
val companyLens: Lens[Employee, Company] = GenLens[Employee](_.company)

companyLens: Lens[Employee, Company] = $sess.cmd6Wrapper$Helper$$anon$1@50875a2e

`GenLens` is a helper scala macro provided by `monocle.macros.GenLens`. A macro is code running at compilation time and emitting code for the actual compilation stage. It emits creation code for our `companyLens` object by introspecting the passed-in selection lambda function. 

Here's what we can now do with our lens:

In [8]:
val foobarCompany = Company("FooBar", Address("Lausanne", Street(58,"rhodanie")))
companyLens.set(foobarCompany)(employee)

foobarCompany: Company = Company(FooBar,Address(Lausanne,Street(58,rhodanie)))
res7_1: Employee = Employee(Monica,Company(FooBar,Address(Lausanne,Street(58,rhodanie))),100)

Note that `Lens.set` is a curried method, i.e. it defines two parameter lists, the first takes the value to be set, the second the object to set it onto. One could also do:

In [9]:
val foobarCompanySetter = companyLens.set(foobarCompany)

foobarCompanySetter: Employee => Employee = <function1>

And we get a function which can be applied to many employees to set their company to foobar:

In [10]:
foobarCompanySetter(employee)
// foobarCompanySetter(bob)
// foobarCompanySetter(paul) 
// etc.

res9: Employee = Employee(Monica,Company(FooBar,Address(Lausanne,Street(58,rhodanie))),100)

Our initial objective actually was to uppercase the street name. Let's see how this can help us get there.

We can define in a similar way lenses for the `address` field of `Company`, for the `street` field of `Address`, and the `name` field of `Street`:

In [11]:
val addressLens    = GenLens[Company](_.address)
val streetLens     = GenLens[Address](_.street)
val streetNameLens = GenLens[Street](_.name)

addressLens: Lens[Company, Address] = $sess.cmd10Wrapper$Helper$$anon$1@2f043ab2
streetLens: Lens[Address, Street] = $sess.cmd10Wrapper$Helper$$anon$2@1342db35
streetNameLens: Lens[Street, String] = $sess.cmd10Wrapper$Helper$$anon$3@7502bcfe

We now have the pieces to **compose** a lens to *zoom* &#128269; into the street name, all the way from the outside of an employee (so to speak &#129325;): 

In [12]:
val employeeStreetNameLens = companyLens composeLens addressLens composeLens streetLens composeLens streetNameLens

employeeStreetNameLens: monocle.PLens[Employee, Employee, String, String] = monocle.PLens$$anon$1@47ac3a7

We can now apply this to our employee!

In [13]:
employeeStreetNameLens.modify(_.capitalize)(employee)

res12: Employee = Employee(Monica,Company(Bestmile,Address(Lausanne,Street(58,Rhodanie))),100)

# &#127870;&#129346;

Here's the same example with `scalaz`:

In [14]:
import scalaz.Lens._
import scalaz.@>

import scalaz.Lens._

import scalaz.@>

In [15]:
val companyL: Employee @> Company = lensu((employee, company) => employee.copy(company = company), _.company)
val addressL: Company @> Address = lensu((company, address) => company.copy(address = address), _.address)
val streetL: Address @> Street = lensu((address, street) => address.copy(street = street), _.street)
val streetNameL: Street @> String = lensu((street, name) => street.copy(name = name), _.name)

companyL: Employee @> Company = scalaz.LensFunctions$$anon$5@1a5c9031
addressL: Company @> Address = scalaz.LensFunctions$$anon$5@5f0fe481
streetL: Address @> Street = scalaz.LensFunctions$$anon$5@5f60d9d8
streetNameL: Street @> String = scalaz.LensFunctions$$anon$5@67fb0ead

Composition is carried out with a fancy `>=>` symbol, `scalaz`-style 	&#128521;:

In [16]:
val employeeStreetNameL = companyL >=> addressL >=> streetL >=> streetNameL

employeeStreetNameL: scalaz.LensFamily[Employee, Employee, String, String] = scalaz.LensFamilyFunctions$$anon$4@585dc122

In [17]:
employeeStreetNameL.mod(_.capitalize, employee)

res16: Employee = Employee(Monica,Company(Bestmile,Address(Lausanne,Street(58,Rhodanie))),100)

Back to `monocle`: we can even zoom-in further than the street name, directly into the `String` type onto the first char. 

Since a `String` can be empty, the first char is optional. We can use `headOption`, a special *optic* for such cases:

In [18]:
import monocle.function.Cons.headOption
(companyLens composeLens addressLens
             composeLens streetLens
             composeLens streetNameLens
             composeOptional headOption).modify(_.toUpper)(employee)

import monocle.function.Cons.headOption

res17_1: Employee = Employee(Monica,Company(Bestmile,Address(Lausanne,Street(58,Rhodanie))),100)

Using `monocle`'s various functions and macros under the `syntax` namespace, one can cut the boilerplate even further:

In [19]:
import monocle.macros.syntax.lens._

employee.lens(_.company.address.street.name).composeOptional(headOption).modify(_.toUpper)

import monocle.macros.syntax.lens._


res18_1: Employee = Employee(Monica,Company(Bestmile,Address(Lausanne,Street(58,Rhodanie))),100)

# Beyond lenses
There are different kinds of optics in `monocle`. They almost all compose with each other. Here's a diagram of all the type, from the most general to the most specific.

![monocle optics](monocle_optics.png "Monocle optics")

## `Iso`: a mirror for immutable structures

`Iso` converts elements of type `S` into elements of type `A` when `S` has the same "shape" as `A`. Consider the following case class:

In [2]:
case class Person(name: String, age: Int)

defined class Person

`Person` is equivalent to a tuple `(String, Int)` and a tuple `(String, Int)` is equivalent to `Person`. We can define "bridges" between types of the same shape that way:

```scala
case class PersonParam(newName: String, newAge: Int)

import monocle.macros.GenIso

val personToTuple = GenIso.fields[Person]
val personParamToTuple = GenIso.fields[PersonParam]
val paramToPerson =  personParamToTuple composeIso personToTuple.reverse
paramToPerson.get(PersonParam("Monica", 21))
// res0: Person = Person(Monica,21)
```

## `Prism` &#128142; : pick a color from the &#127752;

A `Prism` is an optic used to select part of a `Sum` type (also known as `Coproduct`), typically a `sealed trait`.

It's a type with two parameters `Prism[S, A]`: `S` is the `Sum`, `A` a part of the `Sum`.

Here's an example `Command` sealed trait:

In [2]:
case class Coordinates(x: Double, y: Double)

sealed trait Command
case class DriveToPickup(target: Coordinates, passengers: Int) extends Command
case class DriveToStation(stationCoordinates: Coordinates) extends Command
case class DriveToDropoff(target: Coordinates, passengers: Int) extends Command
case object EmergencyStop extends Command
case object EmergencyResume extends Command

defined class Coordinates
defined trait Command
defined class DriveToPickup
defined class DriveToStation
defined class DriveToDropoff
defined object EmergencyStop
defined object EmergencyResume

Now let's say we want to define a generic logging function which logs coordinates inside of commands (if any):

In [8]:
def logAnyCoordinates(command: Command) = ???

defined function logAnyCoordinates

That would be the usual implementation:

In [16]:
def logAnyCoordinatesUsual(command: Command) = {
  val coordinates = 
  command match {
    case DriveToPickup(c, _) => Some(c)
    case DriveToDropoff(c, _) => Some(c)
    case DriveToStation(c) => Some(c)
    case _ => None
  }
  println(coordinates.map(c =>  s"x=${c.x}, y=${c.y}").getOrElse(""))
}

defined function logAnyCoordinatesUsual

In [17]:
logAnyCoordinatesUsual(DriveToDropoff(Coordinates(1.0, 2.0), 2))
logAnyCoordinatesUsual(EmergencyStop)
logAnyCoordinatesUsual(DriveToStation(Coordinates(5.0, 2.0)))
logAnyCoordinatesUsual(EmergencyResume)
logAnyCoordinatesUsual(DriveToDropoff(Coordinates(42, 42), 42))

x=1.0, y=2.0

x=5.0, y=2.0

x=42.0, y=42.0


That's ok, but here are some drawbacks:

- This is evolution-fragile: if the `Sum` changes shape, code needs to be touched.

- This sort of selection code is likely to end up duplicated in various places depending on the needs of each method. 

- It only describes one thing: extracting the value from the `Sum` type 

But most importantly: it's super boring code! &#128564; &#128541;
Let's look at another, more generic way to do this using a combination of `Prism` and `Lens`.

First, we need some imports and a helper method (which unfortunately isn't part of monocle)

In [18]:
import monocle.Optional
import monocle.macros.{GenLens, GenPrism}

implicit class RichOptional[A, B](self: Optional[A, B]) {
  def merge(optional: Optional[A, B]): Optional[A, B] =
    Optional[A, B](a => self.getOption(a).orElse(optional.getOption(a)))(b => a => self.setOption(b)(a).getOrElse(optional.set(b)(a)))
}

import monocle.Optional

import monocle.macros.{GenLens, GenPrism}


defined class RichOptional

We can now define an `Optional` which acts as sort of pointer on all coordinates in the `Command` sealed trait:

In [19]:
val commandCoordinatesOptional = 
GenPrism[Command, DriveToPickup] ^|-> GenLens[DriveToPickup](_.target) merge
GenPrism[Command, DriveToStation] ^|-> GenLens[DriveToStation](_.stationCoordinates) merge
GenPrism[Command, DriveToDropoff] ^|-> GenLens[DriveToDropoff](_.target)

commandCoordinatesOptional: Optional[Command, Coordinates] = monocle.Optional$$anon$6@13b721fb

We can now rewrite our logging function to take advantage of this:

In [21]:
def logAnyCoordinatesCool(command: Command) = {
  println(commandCoordinatesOptional.getOption(command).map(c => s"x=${c.x}, y=${c.y}").getOrElse(""))
}

defined function logAnyCoordinatesCool

In [22]:
logAnyCoordinatesCool(DriveToDropoff(Coordinates(1.0, 2.0), 2))
logAnyCoordinatesCool(EmergencyStop)
logAnyCoordinatesCool(DriveToStation(Coordinates(5.0, 2.0)))
logAnyCoordinatesCool(EmergencyResume)
logAnyCoordinatesCool(DriveToDropoff(Coordinates(42, 42), 42))

x=1.0, y=2.0

x=5.0, y=2.0

x=42.0, y=42.0


Even more &#127378; is the ability to update the coordinates for all! We can now define:

In [27]:
def offsetByOne(coordinates: Coordinates) = coordinates.copy(x = coordinates.x + 1, y = coordinates.y + 1)
def offsetCoordinatesByOne(command: Command) = commandCoordinatesOptional.modify(offsetByOne)(command)

defined function offsetByOne
defined function offsetCoordinatesByOne

In [28]:
offsetCoordinatesByOne(DriveToDropoff(Coordinates(1.0, 2.0), 2))
offsetCoordinatesByOne(EmergencyStop)
offsetCoordinatesByOne(DriveToStation(Coordinates(5.0, 2.0)))
offsetCoordinatesByOne(EmergencyResume)
offsetCoordinatesByOne(DriveToDropoff(Coordinates(42, 42), 42))

res27_0: Command = DriveToDropoff(Coordinates(2.0,3.0),2)
res27_1: Command = EmergencyStop
res27_2: Command = DriveToStation(Coordinates(6.0,3.0))
res27_3: Command = EmergencyResume
res27_4: Command = DriveToDropoff(Coordinates(43.0,43.0),42)

## `Traversal` &#10175;: many eyes see it all 🕷️

`Traversal` can focus into all elements inside of a container (e.g. List, Vector, Option). In more generic terms, it allows to focus from a type `S` into *0 to n* values of type `A`.

For instance, let's say each vehicle keeps a journal 📒 of commands:

In [29]:
case class Vehicle(name: String, id: Int)
case class VehicleCommandsJournal(vehicle: Vehicle, commands: List[Command])

defined class Vehicle
defined class VehicleCommandsJournal

and an instance with some commands:

In [41]:
val journal = VehicleCommandsJournal(
  Vehicle("Navia1", 1),
  List(
    DriveToPickup(Coordinates(1.0, 2.0), 2),
    DriveToDropoff(Coordinates(5.0, 3.0), 2),
    DriveToStation(Coordinates(5.0, 2.0)),
    EmergencyStop
  ))

journal: VehicleCommandsJournal = VehicleCommandsJournal(Vehicle(Navia1,1),List(DriveToPickup(Coordinates(1.0,2.0),2), DriveToDropoff(Coordinates(5.0,3.0),2), DriveToStation(Coordinates(5.0,2.0)), EmergencyStop))

Let's see if we can carry out our coordinates offset stunt. First, we need some more imports:

In [43]:
import monocle.Traversal
import scalaz.std.list.listInstance // we need scalaz Traverse type class instance for List in scope

// code below is only needed to make it work in Jupyter:
import monocle._
import scalaz._  
def fromTraverse[T[_]: Traverse, A, B]: PTraversal[T[A], T[B], A, B] =
    new PTraversal[T[A], T[B], A, B] {
      def modifyF[F[_]: Applicative](f: A => F[B])(s: T[A]): F[T[B]] =
        Traverse[T].traverse(s)(f)
    }

import monocle.Traversal

import scalaz.std.list.listInstance // we need scalaz Traverse type class instance for List in scope

// code below is only needed to make it work in Jupyter:

import monocle._

import scalaz._  

defined function fromTraverse

We can now define a traversal for commands in the journal:

In [39]:
val commandsCoordinatesTraversal = GenLens[VehicleCommandsJournal](_.commands) ^|->> fromTraverse[List, Command, Command]

commandsCoordinatesTraversal: PTraversal[VehicleCommandsJournal, VehicleCommandsJournal, Command, Command] = monocle.PTraversal$$anon$2@479b3e88

We can use this `Traversal` instance to define an offset function which works on `VehicleCommandsJournal`:

In [44]:
def offsetAllCoordinatesByOne(journal: VehicleCommandsJournal) = 
    commandsCoordinatesTraversal.modify(offsetCoordinatesByOne)(journal)

defined function offsetAllCoordinatesByOne

And now let's apply it to our small `journal`:

In [45]:
offsetAllCoordinatesByOne(journal)

res44: VehicleCommandsJournal = VehicleCommandsJournal(Vehicle(Navia1,1),List(DriveToPickup(Coordinates(2.0,3.0),2), DriveToDropoff(Coordinates(6.0,4.0),2), DriveToStation(Coordinates(6.0,3.0)), EmergencyStop))

# 🍻🍻🍻

# Typeclasses
We've seen how to zoom-in on elements based on **structure** using `Lens` and other optics. We can also focus on elements based on **data** using `monocle`'s various typeclasses.

To illustrate this, let's define a structure which maps employees per company:

In [48]:
case class EmployeesPerCompany(employees: Map[Company, List[Employee]])

defined class EmployeesPerCompany

We have two companies around here:

In [49]:
val bm = Company("Bestmile", Address("Lausanne", Street(58, "rhodanie")))
val pmi = Company("Philip Morris International", Address("Lausanne", Street(50, "Rhodanie")))

bm: Company = Company(Bestmile,Address(Lausanne,Street(58,rhodanie)))
pmi: Company = Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie)))

And various employees:

In [51]:
val employeesPerCompany = EmployeesPerCompany(List(
    Employee("Monica", bm, salary = 100),
    Employee("Bob", bm, salary = 90),
    Employee("Alice", bm, salary = 110),
    Employee("Alfred", bm, salary = 105),
    Employee("Picsou", pmi, salary = 1000),
    Employee("Donald", pmi, salary = 2000))
 .groupBy(_.company))

employeesPerCompany: EmployeesPerCompany = EmployeesPerCompany(
  Map(
    Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie))) -> List(
      Employee(Picsou,Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie))),1000),
      Employee(Donald,Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie))),2000)
    ),
    Company(Bestmile,Address(Lausanne,Street(58,rhodanie))) -> List(
      Employee(Monica,Company(Bestmile,Address(Lausanne,Street(58,rhodanie))),100),
...

Notice the injustice  🤑💸? In the world of optics, it's easy to correct this. We first need a lens on the `salary` field of `Employee`:

In [35]:
val salaryLens = GenLens[Employee](_.salary)

salaryLens: Lens[Employee, Int] = $sess.cmd34Wrapper$Helper$$anon$1@150ec40

Then some imports:

In [52]:
import monocle.function.At.at      // to get At Lens
import monocle.function.Each.each  // to get Each Lens
import monocle.std.map._           // to get Map instance for At
import monocle.std.list._          // to get List instance for Each 

import monocle.function.At.at      // to get At Lens

import monocle.function.Each.each  // to get Each Lens

import monocle.std.map._           // to get Map instance for At

import monocle.std.list._          // to get List instance for Each 

We can now do:

In [54]:
val corrected = (employeesPerCompany.lens(_.employees) 
     ^|-> at(pmi) 
     ^|->> each ^|->> each 
     ^|-> salaryLens
).modify(_ / 10)

corrected: EmployeesPerCompany = EmployeesPerCompany(
  Map(
    Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie))) -> List(
      Employee(Picsou,Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie))),100),
      Employee(Donald,Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie))),200)
    ),
    Company(Bestmile,Address(Lausanne,Street(58,rhodanie))) -> List(
      Employee(Monica,Company(Bestmile,Address(Lausanne,Street(58,rhodanie))),100),
...

Of course, let's not forget 😏:

In [55]:
(corrected.lens(_.employees) 
     ^|-> at(bm) 
     ^|->> each ^|->> each 
     ^|-> salaryLens
).modify(_ * 10)

res54: EmployeesPerCompany = EmployeesPerCompany(
  Map(
    Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie))) -> List(
      Employee(Picsou,Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie))),100),
      Employee(Donald,Company(Philip Morris International,Address(Lausanne,Street(50,Rhodanie))),200)
    ),
    Company(Bestmile,Address(Lausanne,Street(58,rhodanie))) -> List(
      Employee(Monica,Company(Bestmile,Address(Lausanne,Street(58,rhodanie))),1000),
...

# For more information
### Documentation:
 - https://julien-truffaut.github.io/Monocle/
 - http://eed3si9n.com/learning-scalaz/Lens.html
 - https://gist.github.com/dragisak/fa8d6c297f20c1348e11
 
### Examples:
 - https://github.com/julien-truffaut/Monocle/tree/master/example/src/test/scala/monocle
 
### Port of monocle for TS 
 - https://github.com/gcanti/monocle-ts 
 
### Exercises: 
 - https://www.scala-exercises.org/monocle/iso